
```python
from enum import Enum

## How I'd like my enum definitions to be declared. `define_enum(...)` is analagous to `attrs.define(...)` which enables specifying various properties alongside the Enum member definitions using a syntax similar to `attrs.field(...)`
# The resultant enum should be comparable and have all the same properties as a native enum.Enum value but in addition have members exposed.
@define_enum(raw_value_type=str)
class PipelineSavingScheme:
	""" 
	PipelineSavingScheme.SKIP_SAVING.shouldSave -> bool >>> False
	PipelineSavingScheme.TEMP_THEN_OVERWRITE.shouldSave -> bool >>> True
	PipelineSavingScheme.OVERWRITE_IN_PLACE.shouldSave -> bool >>> True
	"""
	SKIP_SAVING = enum_member(raw_value="skip_saving", shouldSave=False) # note that `shouldSave` here is not part of the enum definition, it's a property that returns a custom value for this enum_member.
    TEMP_THEN_OVERWRITE = enum_member(raw_value="temp_then_overwrite", shouldSave=True) # saves to a temporary filename if extant exists, then compares and overwrites if needed. Prevents ruining the real pickle if pickling is interrupted/fails.
    OVERWRITE_IN_PLACE = enum_member(raw_value="overwrite_in_place", shouldSave=True)

```


In [ ]:
from enum import Enum

class enum_member:
    def __init__(self, raw_value, **additional_properties):
        self.raw_value = raw_value
        self.additional_properties = additional_properties
        
# ## loses type information, making it an instance of enum.EnumMeta instead of the original class. Native enum.Enum preserves the original class type.
# def define_enum(raw_value_type=str):
#     def wrapper(cls):
#         members = {}
#         additional_properties = {}
#         for name, value in cls.__dict__.items():
#             if isinstance(value, enum_member):
#                 members[name] = value.raw_value
#                 additional_properties[name] = value.additional_properties
#         new_enum = Enum(cls.__name__, members)
#         for name, enum_member_instance in new_enum.__members__.items():
#             for prop_name, prop_value in additional_properties[name].items():
#                 setattr(enum_member_instance, prop_name, prop_value)
#         return new_enum
#     return wrapper

## preserves type information and isinstance
def define_enum(raw_value_type=str):
    def wrapper(cls):
		# Create the Enum class
        enum_class = Enum(cls.__name__, [(name, value.raw_value) for name, value in cls.__dict__.items() if isinstance(value, enum_member)])
        
        # Modify the Enum class to add custom attributes
        for name, value in cls.__dict__.items():
            if isinstance(value, enum_member):
                enum_member_instance = getattr(enum_class, name)
                for prop_name, prop_value in value.additional_properties.items():
                    setattr(enum_member_instance, prop_name, prop_value)

        # Set the Enum class as a subclass of the original class
        cls.__bases__ = (enum_class,)
        return cls
    return wrapper


In [ ]:

@define_enum(raw_value_type=str)
class PipelineSavingScheme:
    SKIP_SAVING = enum_member(raw_value="skip_saving", shouldSave=False)
    TEMP_THEN_OVERWRITE = enum_member(raw_value="temp_then_overwrite", shouldSave=True)
    OVERWRITE_IN_PLACE = enum_member(raw_value="overwrite_in_place", shouldSave=True)

# Example usage
print(PipelineSavingScheme.SKIP_SAVING.shouldSave) # Output: False


In [1]:
from enum import Enum

def define_enum(raw_value_type=str):
    def wrapper(cls):
        # Create an Enum class with the specified members
        enum_class = Enum(cls.__name__, {name: value.raw_value for name, value in cls.__dict__.items() if isinstance(value, enum_member)})

        # Create a new class that inherits from both the original class and the Enum class
        class CombinedEnumClass(cls, enum_class):
            pass

        # Add custom attributes to the new class
        for name, value in cls.__dict__.items():
            if isinstance(value, enum_member):
                enum_member_instance = getattr(CombinedEnumClass, name)
                for prop_name, prop_value in value.additional_properties.items():
                    setattr(enum_member_instance, prop_name, prop_value)

        return CombinedEnumClass

    return wrapper

class enum_member:
    def __init__(self, raw_value, **additional_properties):
        self.raw_value = raw_value
        self.additional_properties = additional_properties

@define_enum(raw_value_type=str)
class PipelineSavingScheme:
    SKIP_SAVING = enum_member(raw_value="skip_saving", shouldSave=False)
    TEMP_THEN_OVERWRITE = enum_member(raw_value="temp_then_overwrite", shouldSave=True)
    OVERWRITE_IN_PLACE = enum_member(raw_value="overwrite_in_place", shouldSave=True)

# Example usage
print(PipelineSavingScheme.SKIP_SAVING.shouldSave) # Output: False


TypeError: CombinedEnumClass: cannot extend enumeration 'PipelineSavingScheme'

In [ ]:



@define_enum(raw_value_type=str)
class PipelineSavingScheme:
    SKIP_SAVING = enum_member(raw_value="skip_saving", shouldSave=False)
    TEMP_THEN_OVERWRITE = enum_member(raw_value="temp_then_overwrite", shouldSave=True)
    OVERWRITE_IN_PLACE = enum_member(raw_value="overwrite_in_place", shouldSave=True)

# Example usage
print(PipelineSavingScheme.SKIP_SAVING.shouldSave) # Output: False
print(isinstance(PipelineSavingScheme.SKIP_SAVING, PipelineSavingScheme)) # Output: True
print(type(PipelineSavingScheme)) # Output: <class '__main__.PipelineSavingScheme'>
